In [ ]:
import requests
import pandas as pd
import time

# API key của bạn từ TMDb
api_key = 'f3a8f89e1d0435429b474be6a159709a'

# Hàm lấy danh sách phim theo năm và sắp xếp theo độ phổ biến
def get_popular_movies(api_key, start_year, end_year, vote_count_threshold=400):
    movies_list = []
    for year in range(start_year, end_year + 1):
        page = 1
        while True:
            url = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&language=en-US&sort_by=popularity.desc&primary_release_year={year}&page={page}&vote_count.gte={vote_count_threshold}'
            response = requests.get(url)
            data = response.json()
            if 'results' not in data or len(data['results']) == 0:
                break
            for movie in data['results']:
                if movie['vote_count'] > vote_count_threshold:
                    movie['year'] = year  # Thêm thông tin năm vào phim
                    movies_list.append(movie)
            page += 1
            time.sleep(0.2)
    return movies_list


In [ ]:
# Hàm lấy thông tin chi tiết của phim
def get_movie_details(api_key, movie_id):
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US&append_to_response=credits'
    response = requests.get(url)
    return response.json()

In [11]:
# Hàm lấy đánh giá của phim
def get_movie_reviews(api_key, movie_id):
    reviews = []
    page = 1
    while True:
        url = f'https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={api_key}&language=en-US&page={page}'
        response = requests.get(url)
        data = response.json()
        if 'results' not in data or len(data['results']) == 0:
            break
        reviews.extend(data['results'])
        page += 1
        time.sleep(0.2)
    return reviews

In [12]:
# Hàm lấy thông tin người dùng
def get_user_details(api_key, user_id):
    url = f'https://api.themoviedb.org/3/user/{user_id}?api_key={api_key}&language=en-US'
    response = requests.get(url)
    return response.json()

In [ ]:
# Lấy danh sách phim phổ biến từ 2017 đến 2024
popular_movies = get_popular_movies(api_key, 2017, 2024)

In [ ]:
# Lấy chi tiết từng phim
detailed_movies = []
for movie in popular_movies:
    details = get_movie_details(api_key, movie['id'])
    movie_data = {
        'id': details['id'],
        'title': details['title'],
        'overview': details.get('overview', ''),
        'tagline': details.get('tagline', ''),
        'vote_count': details.get('vote_count', 0),
        'vote_average': details.get('vote_average', 0.0),
        'popularity': details.get('popularity', 0.0),
        'genres': [genre['name'] for genre in details.get('genres', [])],
        'director': '',
        'actors': [],
        'year': movie['year']  # Lưu lại thông tin năm
    }
    if 'credits' in details:
        crew = details['credits'].get('crew', [])
        directors = [member['name'] for member in crew if member['job'] == 'Director']
        if directors:
            movie_data['director'] = directors[0]
        cast = details['credits'].get('cast', [])
        actors = [member['name'] for member in cast[:5]]  # Lấy 5 diễn viên chính
        movie_data['actors'] = actors
    detailed_movies.append(movie_data)
    time.sleep(0.2)  # Tránh bị chặn bởi server


In [ ]:
# Chuyển đổi dữ liệu thành DataFrame và lưu vào file CSV
movies_df = pd.DataFrame(detailed_movies)
movies_df.to_csv('detailed_movies_2017_2024.csv', index=False)

In [13]:
# Lấy đánh giá của các phim
all_reviews = []
for movie in detailed_movies:
    reviews = get_movie_reviews(api_key, movie['id'])
    for review in reviews:
        review_data = {
            'movie_id': movie['id'],
            'user_id': review['author_details']['username'],
            'rating': review['author_details'].get('rating', None),
            'created_at': review['created_at']
        }
        all_reviews.append(review_data)
    time.sleep(0.2)  # Tránh bị chặn bởi server


In [14]:
# Chuyển đổi dữ liệu đánh giá thành DataFrame
reviews_df = pd.DataFrame(all_reviews)

In [23]:
reviews_df

,movie_id,user_id,rating,created_at
0,284053,Ruuz,8.0,2017-10-25T14:11:35.487Z
1,284053,Movie Queen41,8.0,2017-11-12T21:31:56.063Z
2,284053,tmdb44006625,8.0,2019-03-09T20:43:52.187Z
3,284053,mfrencken,8.0,2019-04-07T04:08:14.999Z
4,284053,ShawnSmith,NaN,2019-11-06T20:42:10.404Z
...,...,...,...,...
800,445040,Bertaut,6.0,2019-04-18T02:43:48.673Z
801,408220,Ruuz,4.0,2017-04-04T15:05:43.977Z
802,411741,Ruuz,7.0,2017-11-08T08:37:47.901Z
803,411741,Rangan,6.0,2018-02-04T13:54:03.461Z


In [25]:
reviews_df.to_csv('reviews_unfiltered.csv', index = False)

In [21]:
# Lọc người dùng có số lượng đánh giá >= 20 và tài khoản tạo trước 2017
user_review_counts = reviews_df['user_id'].value_counts()
filtered_users = user_review_counts[user_review_counts >= 10].index.tolist()

In [22]:
filtered_users

['Ruuz',
 'Rangan',
 'Geronimo1967',
 'r96sk',
 'GenerationofSwine',
 'Dark Jedi',
 'SoSmooth1982',
 'mooney240',
 'John Chard',
 'FilipeManuelNeto',
 'Wuchak']

In [16]:
# Lấy thông tin ngày tạo tài khoản người dùng
user_creation_dates = {}
for user_id in filtered_users:
    user_details = get_user_details(api_key, user_id)
    user_creation_dates[user_id] = user_details.get('created_at', None)
    time.sleep(0.2)  # Tránh bị chặn bởi server

In [18]:
# Lọc người dùng có tài khoản tạo trước 2017
filtered_users = [user_id for user_id, created_at in user_creation_dates.items() if created_at and pd.to_datetime(created_at).year < 2017]

filtered_reviews = reviews_df[reviews_df['user_id'].isin(filtered_users)]

# Lưu dữ liệu đánh giá đã lọc vào file CSV
filtered_reviews.to_csv('filtered_reviews.csv', index=False)

# Kiểm tra kết quả
print(filtered_reviews.head())

Empty DataFrame
Columns: [movie_id, user_id, rating, created_at]
Index: []
